In [3]:
using ModelingToolkit, OrdinaryDiffEq, Plots, DifferentialEquations


@variables t
@connector function Pin(; name)
    sts = @variables v(t)=1.0 i(t)=1.0 [connect = Flow]
    ODESystem(Equation[], t, sts, []; name = name)
end

@component function Ground(; name)
    @named g = Pin()
    eqs = [g.v ~ 0]
    compose(ODESystem(eqs, t, [], []; name = name), g)
end

@component function OnePort(; name)
    @named p = Pin()
    @named n = Pin()
    sts = @variables v(t)=1.0 i(t)=1.0
    eqs = [v ~ p.v - n.v
           0 ~ p.i + n.i
           i ~ p.i]
    compose(ODESystem(eqs, t, sts, []; name = name), p, n)
end
@component function ConstantVoltage(; name, V = 1.0)
    @named oneport = OnePort()
    @unpack v = oneport
    ps = @parameters V = V
    eqs = [
        V ~ v,
    ]
    extend(ODESystem(eqs, t, [], ps; name = name), oneport)
end

@named vtest = ConstantVoltage()
@named g = Ground()
con = connect(g.g,vtest.n)
@named sys = ODESystem(con,t; systems = [vtest,g])
structural_simplify(sys)


Model sys with 0 equations
States (0):
Parameters (1):
  vtest₊V [defaults to 1.0]
Incidence matrix:0×0 SparseArrays.SparseMatrixCSC{Num, Int64} with 0 stored entries

In [9]:
showeq(x) = equations(expand_connections(x))
@component function Resistor(; name, R = 1.0)
    @named oneport = OnePort()
    @unpack v, i = oneport
    ps = @parameters R = R
    eqs = [
        v ~ i * R,
    ]
    extend(ODESystem(eqs, t, [], ps; name = name), oneport)
end

@named mtest = ConstantVoltage()
@named g = Ground()
@named fr = Resistor()
con = [connect(g.g,mtest.n)
        connect(mtest.p,fr.p)]

@named sys = ODESystem(con,t; systems = [mtest,g])
structural_simplify(sys)
showeq(sys)

9-element Vector{Equation}:
 mtest₊v(t) ~ mtest₊p₊v(t) - mtest₊n₊v(t)
 0 ~ mtest₊n₊i(t) + mtest₊p₊i(t)
 mtest₊i(t) ~ mtest₊p₊i(t)
 mtest₊V ~ mtest₊v(t)
 g₊g₊v(t) ~ 0
 g₊g₊v(t) ~ mtest₊n₊v(t)
 0 ~ g₊g₊i(t) + mtest₊n₊i(t)
 mtest₊p₊v(t) ~ fr₊p₊v(t)
 0 ~ fr₊p₊i(t) + mtest₊p₊i(t)

In [12]:

@component function Capacitor(; name, C = 1.0)
    @named oneport = OnePort()
    @unpack v, i = oneport
    ps = @parameters C = C
    D = Differential(t)
    eqs = [
        D(v) ~ i / C,
    ]
    extend(ODESystem(eqs, t, [], ps; name = name), oneport)
end

@component function ConstantVoltage(; name, V = 1.0)
    @named oneport = OnePort()
    @unpack v = oneport
    ps = @parameters V = V
    eqs = [
        V ~ v,
    ]
    extend(ODESystem(eqs, t, [], ps; name = name), oneport)
end

R = 1.0
C = 1.0
V = 1.0
@named resistor = Resistor(R = R)
@named capacitor = Capacitor(C = C)
@named source = ConstantVoltage(V = V)
@named ground = Ground()

rc_eqs = [connect(source.p, resistor.p)
          connect(resistor.n, capacitor.p)
          connect(capacitor.n, source.n)
          connect(capacitor.n, ground.g)]

@named _rc_model = ODESystem(rc_eqs, t)
@named rc_model = compose(_rc_model,
                          [resistor, capacitor, source, ground])
sys = structural_simplify(rc_model)
# u0 = [
#     capacitor.v => 0.0,
# ]
# prob = ODAEProblem(sys, u0, (0, 10.0))
# sol = solve(prob, Tsit5())
# plot(sol)


Model rc_model with 1 equations
States (1):
  capacitor₊v(t) [defaults to 1.0]
Parameters (3):
  resistor₊R [defaults to 1.0]
  capacitor₊C [defaults to 1.0]
  source₊V [defaults to 1.0]
Incidence matrix:1×2 SparseArrays.SparseMatrixCSC{Num, Int64} with 2 stored entries:
 ×  ×

In [192]:
using ModelingToolkit, OrdinaryDiffEq, Plots, DifferentialEquations

@connector function FlowPin(; name)
    sts = @variables T(t)=300.0 P(t)=20.0 ṁ(t)=1.0 [connect = Flow]
    ODESystem(Equation[], t, sts, []; name = name)
end

@component function FlowPort(; name)
    @named fp = FlowPin()
    @named fn = FlowPin()
    sts = @variables T(t) = 1.0 P(t)=1.0 ṁ(t)=1.0
    eqs = [T ~ fp.T - fn.T
            P ~ fp.P - fn.P
            0 ~ fp.ṁ - fn.ṁ
            ṁ ~ fp.ṁ]
    compose(ODESystem(eqs, t, sts, []; name = name), fp, fn)
end


@component function compx(; name, rp = 3.5, cp = 5192.0,η = 0.9, k = 1.667)
    @named flowport = FlowPort()
    @unpack T,P,fp  = flowport
    ps = @parameters CR = rp η = η cp = cp k =k
    sts = @variables Ẇ(t)
    eqs = [
        P ~ fp.P * (rp - 1),
        T ~ fp.T * ((-1 + η + rp^((k-1)/k))/η + 1),
        Ẇ ~ fp.ṁ * cp * T
    ]
    extend(ODESystem(eqs, t, sts, ps; name = name), flowport)
end



┌ Warning: fp contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: fn contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40


SingularException: SingularException(1)

In [ ]:

par = @parameters rp η k cp
# sts = @variables Tin(t) [input = true] Pin(t) [input = true] ΔT(t) ΔP(t) ṁ(t) [input = true] Ẇ(t)
@variables t
D = Differential(t)

State  = @variables Mcv(t) Ecv(t) # change in mass, change in Energy
Input  = @variables ṁ_i(t) Q̇(t) T_i(t) P_i(t)
output = @variables ṁ_o(t) Ẇ(t) T_o(t) P_o(t)

@named CVodesys = ODESystem(eqs,t, [Mcv,Ecv],[])
# @named odeprob = ODAEProblem(odesys,u0,tspan = (0.0,10.0), par)

states(odesys)
# parameters(odesys)
# get_variables(odesys)
# equations(odesys)
u = [ṁ_i, Q̇_i, h_i]
y = [ṁ_o, Ẇ_o, h_o]

Teqs = [
    Q̇_i ~ 0,
    Ẇ_o ~ ṁ_o * h_o - ṁ_i * h_i];
    # h_o ~ h_i *((-1 + η + rp^((k-1)/k))/η)]


alleq = vcat(eqs,Teqs)


using Symbolics
Symbolics.solve_for(alleq, y)
# @show structural_simplify(tsys)
# matrices, simplified_sys = linearize(tsys,u,y) # Linearize from r to y
# states(odesys)